<a href="https://colab.research.google.com/github/We1rdguy/Fastai/blob/master/CNN_with_Pytorch_for_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # to handle matrix and data operation
import pandas as pd # to read csv and handle dataframe

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from torchsummary import summary
from sklearn.model_selection import train_test_split

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
dataset_path="gdrive/My Drive/Dataset/MNIST"
os.chdir(dataset_path)

In [7]:
! ls

digit-recognizer.zip  graph.png  sample_submission.csv	test.csv  train.csv


In [0]:
import pandas as pd

In [9]:
df = pd.read_csv("train.csv")
print(df.shape)

(42000, 785)


In [0]:
y = df['label'].values
X = df.drop(['label'],1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [11]:
print(y_test.shape)

(6300,)


In [0]:
BATCH_SIZE = 32

torch_X_train = torch_X_train.view(-1, 1,28,28).float().cuda()
torch_X_test = torch_X_test.view(-1,1,28,28).float().cuda()

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

In [33]:
print(torch_X_train.shape)
print(torch_X_test.shape)

torch.Size([35700, 1, 28, 28])
torch.Size([6300, 1, 28, 28])


In [36]:

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
    self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
    self.conv3 = nn.Conv2d(32,64, kernel_size=5)
    self.fc1 = nn.Linear(3*3*64, 256)
    self.fc2 = nn.Linear(256, 10)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.dropout(x, p=0.5, training=self.training)
    x = F.relu(F.max_pool2d(self.conv2(x), 2))
    x = F.dropout(x, p=0.5, training=self.training)
    x = F.relu(F.max_pool2d(self.conv3(x),2))
    x = F.dropout(x, p=0.5, training=self.training)
    x = x.view(-1,3*3*64 )
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    return F.log_softmax(x, dim=1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
cnn = CNN().to(device)
summary(cnn, (1, 28, 28))
      

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 24, 24]             832
            Conv2d-2           [-1, 32, 20, 20]          25,632
            Conv2d-3             [-1, 64, 6, 6]          51,264
            Linear-4                  [-1, 256]         147,712
            Linear-5                   [-1, 10]           2,570
Total params: 228,010
Trainable params: 228,010
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.26
Params size (MB): 0.87
Estimated Total Size (MB): 1.13
----------------------------------------------------------------


In [37]:
it = iter(train_loader)
X_batch, y_batch = next(it)
print(cnn.forward(X_batch).shape)

torch.Size([32, 10])


In [0]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters())#,lr=0.001, betas=(0.9,0.999))
    error = nn.CrossEntropyLoss()
    EPOCHS = 5
    model.train()
    for epoch in range(EPOCHS):
      correct = 0
      for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        var_X_batch = Variable(X_batch).float()
        var_y_batch = Variable(y_batch).cuda()
        optimizer.zero_grad()
        output = model(var_X_batch)
        loss = error(output, var_y_batch)
        loss.backward()
        optimizer.step()
        
        # Total correct predictions
        predicted = torch.max(output.data, 1)[1] 
        correct += (predicted == var_y_batch).sum().cuda()
        
        #print(correct)
        if batch_idx % 50 == 0:
          print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
              epoch, 
              batch_idx*len(X_batch), 
              len(train_loader.dataset), 
              100.*batch_idx / len(train_loader), 
              loss.data, 
              float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))

In [70]:
fit(cnn,train_loader)

Epoch : 0 [0/35700 (0%)]	Loss: 0.099395	 Accuracy:96.875%
Epoch : 0 [1600/35700 (4%)]	Loss: 0.108511	 Accuracy:94.363%
Epoch : 0 [3200/35700 (9%)]	Loss: 0.104907	 Accuracy:94.616%
Epoch : 0 [4800/35700 (13%)]	Loss: 0.204859	 Accuracy:94.495%
Epoch : 0 [6400/35700 (18%)]	Loss: 0.382906	 Accuracy:94.574%
Epoch : 0 [8000/35700 (22%)]	Loss: 0.005160	 Accuracy:94.646%
Epoch : 0 [9600/35700 (27%)]	Loss: 0.122724	 Accuracy:94.539%
Epoch : 0 [11200/35700 (31%)]	Loss: 0.189900	 Accuracy:94.489%
Epoch : 0 [12800/35700 (36%)]	Loss: 0.263274	 Accuracy:94.490%
Epoch : 0 [14400/35700 (40%)]	Loss: 0.608493	 Accuracy:94.374%
Epoch : 0 [16000/35700 (45%)]	Loss: 0.087857	 Accuracy:94.380%
Epoch : 0 [17600/35700 (49%)]	Loss: 0.158362	 Accuracy:94.323%
Epoch : 0 [19200/35700 (54%)]	Loss: 0.172437	 Accuracy:94.317%
Epoch : 0 [20800/35700 (58%)]	Loss: 0.508515	 Accuracy:94.230%
Epoch : 0 [22400/35700 (63%)]	Loss: 0.307453	 Accuracy:94.316%
Epoch : 0 [24000/35700 (67%)]	Loss: 0.229471	 Accuracy:94.337%
Epoch

In [0]:
def evaluate(model):
  correct = 0
  for test_imgs, test_labels in test_loader:
    #print(test_imgs.shape)
    test_imgs = Variable(test_imgs).float()
    output = model(test_imgs)
    predicted = torch.max(output,1)[1]
    correct += (predicted == test_labels).sum()
    
  print("Test accuracy:{:.3f}% ".format( float(correct) / (len(test_loader)*BATCH_SIZE)))  

In [72]:
evaluate(cnn)

RuntimeError: ignored